### ЗНАКОМСТВО С ДАТАСЕТОМ

Напоминаем, что датасет, с которым мы будем работать, уже вам знаком. Мы взяли те же данные HeadHunter, только разбили их по таблицам и перенесли в БД SQL. Такой процесс называется нормализацией (вы можете почитать про неё дополнительно, если вам понадобятся знания по проектированию баз данных).

Мы видим четыре таблицы:

#### CANDIDATE

Таблица хранит в себе общие данные по кандидатам: id, пол, возраст, желаемая должность, город, вид занятости, текущая должность, дата обновления записи и зарплата.

#### CITY

city — таблица-справочник для наших кандидатов — хранит код города и его название.

#### CANDIDATE_TIMETABLE_TYPE

Это дополнительная таблица. Она существует для организации связи многие-ко-многим, так как у нас есть много кандидатов и у них может быть несколько подходящих типов рабочего графика.

Примечание. Нам стоило бы вынести в дополнительную таблицу и колонку employment_type из таблицы candidate, так как вид связи здесь одинаковый, но мы решили оставить всё как есть, так как это даст нам дополнительные возможности для постановки интересных задач и проведения изысканий.

#### TIMETABLE_TYPE

Это таблица-справочник вариантов рабочего графика, подходящего кандидату.



✍ Для начала давайте познакомимся с общей информацией по кандидатам и попытаемся оценить адекватность данных. Начнём с возраста.

Задание 2.1

Рассчитайте максимальный возраст (max_age) кандидата в таблице.

    SELECT
        MAX(c.age) max_age
    FROM 
        hh.CANDIDATE c




Задание 2.2

Теперь давайте рассчитаем минимальный возраст (min_age) кандидата в таблице

    SELECT
        MIN(c.age) min_age
    FROM 
        hh.CANDIDATE c

Задание 2.3

Попробуем «почистить» данные. Напишите запрос, который позволит посчитать для каждого возраста (age) сколько (cnt) человек этого возраста у нас есть.
Отсортируйте результат по возрасту в обратном порядке.

    SELECT
        c.age age,
        COUNT (c.id) cnt
    FROM 
        hh.CANDIDATE c
    GROUP BY c.age
    ORDER BY 1 DESC



Задание 2.4

По данным Росстата, средний возраст занятых в экономике России составляет 39.7 лет. Мы округлим это значение до 40. Найдите количество кандидатов, которые старше данного возраста. Не забудьте отфильтровать «ошибочный» возраст 100.

    SELECT
        COUNT (c.id) cnt
    FROM 
        hh.CANDIDATE c
    WHERE    
        c.age > 40
        AND c.age !=100



### 3. Глобальный анализ показателей

✍ Теперь нам хотелось бы получить общие показатели по данным.

Задание 3.1

Для начала напишите запрос, который позволит узнать, сколько (cnt) у нас кандидатов из каждого города (city).
Формат выборки: city, cnt.
Группировку таблицы необходимо провести по столбцу title, результат отсортируйте по количеству в обратном порядке.

    SELECT 
        ct.title city,
        COUNT(cd.id) cnt 
    FROM
        hh.CANDIDATE cd
        JOIN hh.city ct ON cd.city_id = ct.id
    GROUP BY city
    ORDER BY 2 DESC

Задание 3.2

Москва бросается в глаза как, пожалуй, самый активный рынок труда. Напишите запрос, который позволит понять, каких кандидатов из Москвы устроит «проектная работа».
Формат выборки: gender, age, desirable_occupation, city, employment_type.
Отсортируйте результат по id кандидата

    SELECT
        cd.gender gender,
        cd.age age,
        cd.desirable_occupation desirable_occupation,
        ct.title city,
        cd.employment_type employment_type
    FROM hh.CANDIDATE cd
        JOIN hh.city ct ON cd.city_id = ct.id
    WHERE (ct.title = 'Москва') AND (cd.employment_type LIKE '%проектная работа%')
    ORDER BY cd.id



Задание 3.3

Данных оказалось многовато. Отфильтруйте только самые популярные IT-профессии — разработчик, аналитик, программист.
Обратите внимание, что данные названия могут быть написаны как с большой, так и с маленькой буквы.
Отсортируйте результат по id кандидата.

    SELECT
        cd.gender gender,
        cd.age age,
        cd.desirable_occupation desirable_occupation,
        ct.title city,
        cd.employment_type employment_type
    FROM hh.CANDIDATE cd
        JOIN hh.city ct ON cd.city_id = ct.id
    WHERE (ct.title = 'Москва') AND (cd.employment_type LIKE '%проектная работа%') AND (lower(desirable_occupation) LIKE '%разработчик%' OR desirable_occupation LIKE '%аналитик%' OR desirable_occupation LIKE '%программист%')
    ORDER BY cd.id



Задание 3.4

Для общей информации попробуйте выбрать номера и города кандидатов, у которых занимаемая должность совпадает с желаемой.
Формат выборки: id, city.
Отсортируйте результат по городу и id кандидата.

    SELECT
        cd.id,
        ct.title city
    FROM hh.CANDIDATE cd
        JOIN hh.city ct ON cd.city_id = ct.id
    WHERE cd.current_occupation = cd.desirable_occupation 
    ORDER BY 2,1



Задание 3.5

Определите количество кандидатов пенсионного возраста.
Пенсионный возраст для мужчин наступает в 65 лет, для женщин — в 60 лет.

    SELECT
        COUNT(c.id) 
    FROM
        hh.CANDIDATE c
    WHERE (c.gender = 'F' AND c.age>=60) OR (c.gender = 'M' AND c.age>=65) AND c.age !=100




Задание 4.1

Для добывающей компании нам необходимо подобрать кандидатов из Новосибирска, Омска, Томска и Тюмени, которые готовы работать вахтовым методом.
Формат выборки: gender, age, desirable_occupation, city, employment_type, timetable_type.
Отсортируйте результат по городу и номеру кандидата.


    SELECT
        cd.gender,
        cd.age,
        cd.desirable_occupation,
        ct.title city,
        cd.employment_type,
        tt.title timetable_type
    FROM hh.CANDIDATE cd
        JOIN hh.city ct ON cd.city_id = ct.id
        JOIN hh.candidate_timetable_type ctt ON cd.id = ctt.candidate_id
        JOIN hh.timetable_type tt ON ctt.timetable_id = tt.id
    WHERE (ct.title IN ('Новосибирск', 'Омск', 'Томск', 'Тюмень')) AND (tt.title ='вахтовый метод')
    ORDER BY 4, cd.id
    

Задание 4.2

Для заказчиков из Санкт-Петербурга нам необходимо собрать список из 10 желаемых профессий кандидатов из того же города от 16 до 21 года (в выборку включается 16 и 21, сортировка производится по возрасту) с указанием их возраста, а также добавить строку Total с общим количеством таких кандидатов.

    (SELECT
        cd.desirable_occupation,
        cd.age age
    FROM hh.CANDIDATE cd
        JOIN hh.city ct ON cd.city_id = ct.id  
    WHERE (ct.title = 'Санкт-Петербург') AND (cd.age BETWEEN 16 AND 21)
    ORDER BY 2
    LIMIT 10)

    UNION ALL

    (SELECT
        'Total', COUNT(cd.id)
    FROM
        hh.CANDIDATE cd
        JOIN hh.city ct ON cd.city_id = ct.id 
    WHERE (ct.title = 'Санкт-Петербург') AND (cd.age BETWEEN 16 AND 21))
 
